<a href="https://colab.research.google.com/github/erlonL/series-temporais/blob/main/ST_Projeto2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# !pip install yfinance snscrape pandas_market_calendars pandas_datareader

In [32]:
import yfinance as yf
#import snscrape.modules.twitter as sntwitter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_market_calendars as mcal
import pandas_datareader.data as web
import pickle
import snscrape
from transformers import pipeline

In [33]:
new_model = pipeline(
    model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    return_all_scores=True
)

/home/erlo/.local/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/home/erlo/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## ETAPA 1: Coleta e Tratamento de Dados

S&P 500, abreviação de Standard & Poor's 500, ou simplesmente S&P, trata-se de um índice composto por quinhentos ativos cotados nas bolsas de NYSE ou NASDAQ, qualificados devido ao seu tamanho de mercado, sua liquidez e sua representação de grupo industrial.

### Dados do mercado de ações

In [34]:
# ticker S&P 500
ticker = '^GSPC'
index = yf.Ticker(ticker)

# de acordo com nossa escolha
start_date = '2020-01-01'
end_date = '2024-09-25'

data = index.history(start=start_date, end=end_date)

# deixar apenas a data (y/m/d) na coluna date
data.index = data.index.date

# Exiba os primeiros registros para verificação
print(data.head())

                   Open         High          Low        Close      Volume  \
2020-01-02  3244.669922  3258.139893  3235.530029  3257.850098  3459930000   
2020-01-03  3226.360107  3246.149902  3222.340088  3234.850098  3484700000   
2020-01-06  3217.550049  3246.840088  3214.639893  3246.280029  3702460000   
2020-01-07  3241.860107  3244.909912  3232.429932  3237.179932  3435910000   
2020-01-08  3238.590088  3267.070068  3236.669922  3253.050049  3726840000   

            Dividends  Stock Splits  
2020-01-02        0.0           0.0  
2020-01-03        0.0           0.0  
2020-01-06        0.0           0.0  
2020-01-07        0.0           0.0  
2020-01-08        0.0           0.0  


No artigo, foi filtrado os dados para os dias úteis do mercado. Faremos a seguir esse processamento:

In [35]:
# filtrando os dados de stock para os dias uteis
nyse = mcal.get_calendar('NYSE')

dias_uteis = nyse.valid_days(start_date=start_date, end_date=end_date)

# Pegar so a data
dias_uteis = [dia.date() for dia in dias_uteis]

In [36]:
sp500 = data.loc[data.index.intersection(dias_uteis)]

print(sp500.head())

sp500.to_csv("data/csv/sp500.csv")

                   Open         High          Low        Close      Volume  \
2020-01-02  3244.669922  3258.139893  3235.530029  3257.850098  3459930000   
2020-01-03  3226.360107  3246.149902  3222.340088  3234.850098  3484700000   
2020-01-06  3217.550049  3246.840088  3214.639893  3246.280029  3702460000   
2020-01-07  3241.860107  3244.909912  3232.429932  3237.179932  3435910000   
2020-01-08  3238.590088  3267.070068  3236.669922  3253.050049  3726840000   

            Dividends  Stock Splits  
2020-01-02        0.0           0.0  
2020-01-03        0.0           0.0  
2020-01-06        0.0           0.0  
2020-01-07        0.0           0.0  
2020-01-08        0.0           0.0  


In [37]:
sp500.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,1190.000000,1190.000000,1190.000000,1190.000000,1.190000e+03,1190.0,1190.0
mean,4165.217473,4190.424592,4138.559578,4166.022263,4.403520e+09,0.0,0.0
std,677.736933,675.675810,680.305417,677.989815,1.049796e+09,0.0,0.0
min,2290.709961,2300.729980,2191.860107,2237.399902,1.639500e+09,0.0,0.0
25%,3786.674988,3819.482483,3749.492493,3790.517395,3.762062e+09,0.0,0.0
50%,4170.040039,4193.760010,4142.614990,4170.560059,4.130170e+09,0.0,0.0
75%,4531.830200,4545.984863,4509.594971,4527.872559,4.748670e+09,0.0,0.0
max,5727.660156,5735.319824,5704.220215,5732.930176,9.976520e+09,0.0,0.0


### Dados de Notícias / Redes Sociais

In [38]:
# news articles
with open('data/pkl/df_news.pkl', 'rb') as f:
    news = pickle.load(f)
# bsky tweets
with open('data/pkl/df_socials.pkl', 'rb') as f:
    bsky = pickle.load(f)

In [39]:
news.head(2)

,title,text,label,links,symbol,company,date
0,Stocks making the biggest moves midday: Palo A...,A view of the exterior of the new Dutch head o...,-2,https://www.cnbc.com/2023/06/05/stocks-making-...,MMM,3M,2023-06-05
1,3M will spin off its health-care business into...,3M announced Tuesday it will spin off its heal...,-2,https://www.cnbc.com/2022/07/26/3m-will-spin-o...,MMM,3M,2022-07-26


In [40]:
bsky.head(2)

,name,user,date,text
0,Roberto Marsicano,@cannedcat.bsky.social,2024-09-21,a top-1% fund manager warns warren buffett is ...
1,copgamer,@copigamer.bsky.social,2024-09-21,microstrategy outperformed the s&p 500 with it...


# Etapa 2: Análise e Agregação de Sentimento através do DistilRoBERTa

In [41]:
# diminuir o número de sentenças de cada texto
'''
import requests
import json
import numpy as np

url = 'https://issam9-sumy-space.hf.space/api/predict/'
data = []
data = data + ['luhn'] + ['english'] + ['1'] + ['text']
data = data + [news['text'].iloc[0]]

json = {
    "cleared": False,
    "data": data,
    "example_id": None,
    "session_hash": "cb1l3gk9k7s"
}

news.loc[:, 'short_text'] = news['text']

for index, row in news.iterrows():
    data[4] = row['text']
    sentiments = {}
    sentiments['negative'] = [] 
    sentiments['neutral'] = []
    sentiments['positive'] = []

    for i in range(8, 15):
        try:
            data[2] = str(i)
            json['data'] = data
            
            response = requests.post(url, json=json)

            responseDICT = response.json()
            text = responseDICT['data'][0]
            print(f'iteração {i}')
            print(f'{index} of {len(news)}')
            # tentar aplicar o modelo
            result = new_model(text)

            sentiments['negative'].append(result[0][0]['score'])
            sentiments['neutral'].append(result[0][1]['score'])
            sentiments['positive'].append(result[0][2]['score'])

        except:
            print(f'Máximo de sentenças: {i}')
            news.loc[index, 'short_text'] = text
            print(result)
            # calcular a mediana dos sentimentos
            news.loc[index, 'sentiment_negative'] = np.median(sentiments['negative'])
            news.loc[index, 'sentiment_neutral'] = np.median(sentiments['neutral'])
            news.loc[index, 'sentiment_positive'] = np.median(sentiments['positive'])
            break
'''

'\nimport requests\nimport json\nimport numpy as np\n\nurl = \'https://issam9-sumy-space.hf.space/api/predict/\'\ndata = []\ndata = data + [\'luhn\'] + [\'english\'] + [\'1\'] + [\'text\']\ndata = data + [news[\'text\'].iloc[0]]\n\njson = {\n    "cleared": False,\n    "data": data,\n    "example_id": None,\n    "session_hash": "cb1l3gk9k7s"\n}\n\nnews.loc[:, \'short_text\'] = news[\'text\']\n\nfor index, row in news.iterrows():\n    data[4] = row[\'text\']\n    sentiments = {}\n    sentiments[\'negative\'] = [] \n    sentiments[\'neutral\'] = []\n    sentiments[\'positive\'] = []\n\n    for i in range(8, 15):\n        try:\n            data[2] = str(i)\n            json[\'data\'] = data\n            \n            response = requests.post(url, json=json)\n\n            responseDICT = response.json()\n            text = responseDICT[\'data\'][0]\n            print(f\'iteração {i}\')\n            print(f\'{index} of {len(news)}\')\n            # tentar aplicar o modelo\n            result

In [42]:
news['text'].iloc[0]

'A view of the exterior of the new Dutch head office of international technology company 3M in Delft, Netherlands, November 5, 2014.\n\nCheck out the companies making headlines in midday trading.\n\nPalo Alto Networks — The cybersecurity stock jumped more than 4% after S&P Dow Jones Indices announced Friday postmarket it will replace Dish Network in the S&P 500, effective June 20. Dish Network dipped about 1%.\n\n3M — The industrial manufacturer\'s shares slid 3% after the judge in the company\'s multidistrict litigation over so-called forever chemicals agreed to delay the first trial three weeks so parties can finalize a settlement of claims with municipal water providers, Bloomberg reported Monday.\n\nCoinbase — Shares of the crypto exchange and services company tumbled 10% after the U.S. Securities and Exchange Commission sued crypto exchange Binance on Monday, alleging Binance and its co-founder Changpeng Zhao commingled billions of dollars of investor funds with their own and viol

In [43]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import RobertaTokenizer, TFRobertaModel
from tqdm import tqdm
# import clear_output()
from IPython.display import clear_output

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('portuguese'))

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = TFRobertaModel.from_pretrained("roberta-base")

def remove_stopwords(text):
    words = text.split()
    return " ".join([word for word in words if word.lower() not in stop_words])

def tokenize(text):
    tokenizer(text, truncation=True, max_length=500, return_tensors='tf')

clear_output()

In [44]:
news.head()

,title,text,label,links,symbol,company,date
0,Stocks making the biggest moves midday: Palo A...,A view of the exterior of the new Dutch head o...,-2,https://www.cnbc.com/2023/06/05/stocks-making-...,MMM,3M,2023-06-05
1,3M will spin off its health-care business into...,3M announced Tuesday it will spin off its heal...,-2,https://www.cnbc.com/2022/07/26/3m-will-spin-o...,MMM,3M,2022-07-26
2,"Stocks making the biggest moves midday: 3M, Pa...",A woman walks near a Bed Bath & Beyond branch ...,0,https://www.cnbc.com/2023/01/24/stocks-making-...,MMM,3M,2023-01-24
3,"3M to cut 2,500 jobs as demand weakens, profit...",An employee unboxes N95 masks as part of a gov...,0,https://www.cnbc.com/2023/01/24/3m-to-cut-2500...,MMM,3M,2023-01-24
4,"3M combat earplug lawsuits to proceed, judge r...","3M must face more than 230,000 lawsuits accusi...",-2,https://www.cnbc.com/2022/08/26/3m-combat-earp...,MMM,3M,2022-08-26


In [49]:
tqdm.pandas()

tokens = []
for index, row in news.iterrows():
    tokens.append(tokenizer(row['text'], truncation=True, max_length=500, return_tensors='tf'))

decoded = []
for token in tokens:
    decoded.append(tokenizer.decode(token['input_ids'][0]))

news['decoded'] = decoded
news['decoded'] = news['decoded'].apply(remove_stopwords)

news.loc[:, 'short_text'] = news['decoded']
news.loc[:, 'sentiment'] = news['short_text'].progress_apply(lambda x: new_model(x))

news.head()

  6%|▌         | 143/2411 [00:39<10:25,  3.63it/s]


KeyboardInterrupt: 